In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import numpy as np
import tensorflow as tf
import tqdm
import io
import imageio

from collections import deque
from skimage.transform import resize, rescale
# from sewar.full_ref import msssim, ssim
from skimage.measure import compare_ssim as ssim

import matplotlib.pyplot as plt

# Own libraries and modules
from helpers import loading, plotting, utils, summaries, tf_helpers
from models import compression

In [ ]:
training = {
    'n_epochs': 5000,
    'batch_size': 40,
    'patch_size': 128
}

In [ ]:
patch_size = 128
n_latent = 1024
n_latent_bytes = 0.5

bitmap_size = patch_size * patch_size * 3
compression_rate = bitmap_size / (n_latent_bytes * n_latent)
compression_bpp = 8 * n_latent * n_latent_bytes / patch_size / patch_size

print('Bitmap size: {:,d} bytes'.format(bitmap_size))
print('Latent size: {:,}-D'.format(n_latent))
print('Latent repr: {:,} bytes'.format(n_latent * n_latent_bytes))
print('Compression rate: 1:{}'.format(compression_rate))
print('Compression Fi  : {:.2f} bpp'.format(compression_bpp))

## Load the dataset

In [ ]:
class IPDataset:
    
    def __init__(self, data_directory, randomize=False, load='xy', val_patch_size=128, val_n_patches=2, n_images=120, v_images=30):
        self.files = {}
        self.files['training'], self.files['validation'] = loading.discover_files(data_directory, randomize=randomize, n_images=n_images, v_images=v_images)
                
        self.data = {
            'training': loading.load_images(self.files['training'], data_directory=data_directory, load=load),
            'validation': loading.load_patches(self.files['validation'], data_directory=data_directory, patch_size=val_patch_size // 2, n_patches=val_n_patches, load=load, discard_flat=True)
        }
        
        if 'x' in self.data['training']:
            self.H, self.W = self.data['training']['x'].shape[1:3]
        else:
            self.H, self.W = self.data['training']['y'].shape[1:3]
            
        print('Loaded dataset with {}x{} images'.format(self.W, self.H))
        
    def __getitem__(self, key):
        if key in ['training', 'validation']:
            return self.data[key]
        else:
            return super().__getitem__(key)
        
    def next_training_batch(self, batch_id, batch_size, patch_size, discard_flat=True):
        batch_x = np.zeros((batch_size, patch_size, patch_size, 3), dtype=np.float32)
        for b in range(batch_size):
            
            found = False            
            panic_counter = 5
            
            while not found:
                xx = np.random.randint(0, self.W - patch_size)
                yy = np.random.randint(0, self.H - patch_size)

                patch = self.data['training']['y'][batch_id * batch_size + b, yy:yy + patch_size, xx:xx + patch_size, :].astype(np.float) / (2**8 - 1)

                # Check if the found patch is acceptable: eliminate empty patches
                if discard_flat:
                    patch_variance = np.var(patch)
                    if patch_variance < 1e-2:
                        panic_counter -= 1
                        found = False if panic_counter > 0 else True
                    elif patch_variance < 0.02:
                        found = np.random.uniform() > 0.5
                    else:
                        found = True
                else:
                    found = True
            
            batch_x[b, :, :, :] = patch
        return batch_x

    def next_validation_batch(self, batch_id, batch_size, output_patch_size=None):
        patch_size = self.data['validation']['y'].shape[1]
        batch_x = np.zeros((batch_size, patch_size, patch_size, 3), dtype=np.float32)
        for b in range(batch_size):
            batch_x[b, :, :, :] = self.data['validation']['y'][batch_id * batch_size + b].astype(np.float)
        if output_patch_size is None or output_patch_size == patch_size:
            return batch_x
        else:
            return batch_x[:, :output_patch_size, :output_patch_size, :]

In [ ]:
# Load data
# camera_name = "Nikon D90"
# data_directory = os.path.join('./data/raw/nip_training_data/', camera_name)
data_directory = os.path.join('./data/raw/compression_data/')

# data = IPDataset(data_directory, n_images=120, v_images=30, load='y')
# data = IPDataset(data_directory, n_images=16000, v_images=800, load='y', val_patch_size=training['patch_size'])
data = IPDataset(data_directory, n_images=4000, v_images=200, load='y', val_patch_size=training['patch_size'])

for dataset in ['training', 'validation']:
    print('{} : {}'.format(dataset, data[dataset]['y'].shape))

In [ ]:
# Show sample batch from the database
if 'batch_id' not in globals():
    batch_id = 0
    n_batches = data['training']['y'].shape[0] // training['batch_size']
    
batch_id = (batch_id + 1) % n_batches

print('Batch id: {} ({})'.format(batch_id, training['batch_size']))

batch_x = data.next_training_batch(batch_id, training['batch_size'], training['patch_size'])
fig = plotting.imsc(batch_x[:8], ncols=8, figwidth=20)

## Define the Deep Compression Network Models

In [ ]:
class AutoencoderDCN(compression.DCN):
    
    def construct_model(self, *, n_filters=8, n_fscale=2, n_latent=0, kernel=5, n_layers=3, r_layers=0, dropout=True, rounding='soft', 
                        is_training=True, train_codebook=False, latent_bpf=8, scale_latent=True, activation=tf.nn.leaky_relu, entropy_weight=None):
        
        # Sanity checks:
        if n_layers < 1:
            raise ValueError('n_layers needs to be > 0!')
        
        self.n_layers = n_layers
        self.r_layers = r_layers
        self.n_latent = n_latent
        self.n_filters = n_filters
        self.n_fscale = n_fscale
        self.kernel = kernel
        self.is_training = is_training
        self.train_codebook = train_codebook
        self.scale_latent = scale_latent
        self.entropy_weight = entropy_weight
        self.latent_bpf = latent_bpf
        self.uses_bottleneck = n_latent > 0
        
        latent_activation = None
        last_activation = None

        print('Building Deep Compression Network')

        net = self.x
        print('in size: {}'.format(net.shape))

        # Encoder ---------------------------------------------------------------------------------------------------------------
        
        # Add convolutional layers
        n_filters = self.n_filters

        for r in range(self.n_layers):
            current_activation = activation if (n_latent > 0 or (n_latent == 0 and r < self.n_layers - 1)) else latent_activation
            net = tf.contrib.layers.conv2d(net, n_filters, self.kernel, stride=2, scope='dcn{}/encoder/conv_{}'.format(self.label, r), activation_fn=current_activation)
            print('conv size: {} + {}'.format(net.shape, current_activation.__name__ if current_activation is not None else None))
            if r != self.n_layers - 1:
                n_filters *= self.n_fscale
            
        # Add residual blocks
        for r in range(self.r_layers):
            resnet = tf.contrib.layers.conv2d(tf.nn.leaky_relu(net, name='dcn{}/encoder/res_{}/lrelu'.format(self.label, r)),    n_filters, 3, stride=1, activation_fn=activation, scope='dcn{}/encoder/res_{}/conv_{}'.format(self.label, r, 0))
            resnet = tf.contrib.layers.conv2d(resnet, n_filters, 3, stride=1, activation_fn=None,       scope='dcn{}/encoder/res_{}/conv_{}'.format(self.label, r, 1))
            net = tf.add(net, resnet, name='dcn{}/encoder/res_{}/sum'.format(self.label, r))
            print('residual block: {}'.format(net.shape))        

        # Latent representation -----------------------------------------------------------------------------------------------------------
            
        # Add batch norm to normalize the latent representation
        if is_training is not None:            
            net = tf.contrib.layers.batch_norm(net, scale=True, is_training=is_training, scope='dcn{}/encoder/bn_{}'.format(self.label, 0))
            print('batch norm: {} train:{}'.format(net.shape, is_training))
        
        # Flatten and get latent representation
        z_spatial = int(self.patch_size / (2**self.n_layers))
        z_features = int(self.n_filters * (self.n_fscale**(self.n_layers-1)))
        self.latent_shape = [-1, z_spatial, z_spatial, z_features]

        # If a smaller linear bottleneck is specified explicitly - add dense layers to make the projection
        if n_latent is not None and n_latent != 0:
            flat = tf.contrib.layers.flatten(net, scope='dcn{}/encoder/flatten_{}'.format(self.label, 0))
            print('flatten size: {}'.format(flat.shape))
            
            if n_latent > 0:
                flat = tf.contrib.layers.fully_connected(flat, self.n_latent, activation_fn=latent_activation, scope='dcn{}/encoder/dense_{}'.format(self.label, 0))
                latent = tf.identity(flat, name='dcn{}/latent'.format(self.label))
                print('dense size: {}'.format(flat.shape))
            else:
                latent = tf.identity(flat, name='dcn{}/latent'.format(self.label))                
        else:
            latent = tf.identity(net, name='dcn{}/latent'.format(self.label))
        
        # Learn a scaling factor for the latent features to encourage greater values (facilitates quantization)
        if self.scale_latent:
            alphas = tf.get_variable('dcn{}/latent_scaling'.format(dcn.label), shape=(), dtype=tf.float32, initializer=tf.ones_initializer)
            latent = tf.multiply(alphas, latent, name='dcn{}/latent_scaled'.format(self.label))            

        # Quantize the latent representation and remember tensors before and after the process
        self.latent_pre = latent
        latent = tf_helpers.quantization(latent, 'dcn{}/quantization'.format(self.label), 'latent_quantized', rounding)                        
        self.latent_post = latent
        self.n_latent = int(np.prod(latent.shape[1:]))
        print('latent size: {} + quant:{}'.format(latent.shape, rounding))
        
        if n_latent > 0:
            inet = tf.contrib.layers.fully_connected(latent, int(np.prod(self.latent_shape[1:])), activation_fn=activation, scope='dcn{}/decoder/dense_{}'.format(self.label, 0))
            print('dense size: {} + {}'.format(inet.shape, activation))
        else:
            inet = latent

        # Add dropout
        if dropout > 0:
            self.dropout = tf.placeholder(tf.float32, name='dcn{}/droprate'.format(self.label), shape=())
            inet = tf.contrib.layers.dropout(inet, keep_prob=self.dropout, scope='dcn{}/dropout'.format(self.label))
            print('dropout size: {}'.format(net.shape))
            
        # Decoder ---------------------------------------------------------------------------------------------------------------
        
        # Just in case - make sure we have a multidimensional tensor before we start the convolutions
        inet = tf.reshape(inet, self.latent_shape, name='dcn{}/decoder/reshape_{}'.format(self.label, 0))
        print('reshape size: {}'.format(inet.shape))

        # Add residual blocks
        for r in range(self.r_layers):
            resnet = tf.contrib.layers.conv2d(tf.nn.leaky_relu(inet, name='dcn{}/encoder/res_{}/lrelu'.format(self.label, r)),   n_filters, 3, stride=1, activation_fn=activation, scope='dcn{}/decoder/res_{}/conv_{}'.format(self.label, r, 0))
            resnet = tf.contrib.layers.conv2d(resnet, n_filters, 3, stride=1, activation_fn=None,       scope='dcn{}/decoder/res_{}/conv_{}'.format(self.label, r, 1))
            inet = tf.add(inet, resnet, name='dcn{}/decoder/res_{}/sum'.format(self.label, r))
            print('residual block: {}'.format(net.shape))                
        
        # Transposed convolutions
        for r in range(self.n_layers):            
            current_activation = last_activation if r == self.n_layers - 1 else activation
            inet = tf.contrib.layers.conv2d(inet, 2 * n_filters, self.kernel, stride=1, scope='dcn{}/decoder/tconv_{}'.format(self.label, r), activation_fn=current_activation)
            print('conv size: {} + {}'.format(inet.shape, current_activation.__name__ if current_activation is not None else None))
            inet = tf.depth_to_space(inet, 2, name='dcn{}/decoder/d2s_{}'.format(self.label, r))
#             inet = tf.contrib.layers.conv2d_transpose(inet, 3 if r == self.n_layers - 1 else n_filters, self.kernel, stride=2,  activation_fn=current_activation, scope='dcn{}/tconv_{}'.format(self.label, r))
            print('d2s size: {} + {}'.format(inet.shape, None))
            n_filters = n_filters // self.n_fscale

        inet = tf.contrib.layers.conv2d(inet, 3, self.kernel, stride=1, activation_fn=last_activation, scope='dcn{}/decoder/tconv_out'.format(self.label))
        print('conv->out size: {} + {}'.format(inet.shape, last_activation))
        y = tf.identity(inet, name='y')
            
        self.y = y
        self.latent = latent
    
    def short_name(self):
        parameter_summary = []
        
        if hasattr(self, 'latent_shape'):
            parameter_summary.append('x'.join(str(x) for x in self.latent_shape[1:]))

        layer_summary = []
        if hasattr(self, 'n_layers'):
            layer_summary.append('{:d}C'.format(self.n_layers))
        if hasattr(self, 'res_layers'):
            layer_summary.append('{:d}R'.format(self.res_layers))
        if self.uses_bottleneck:
            layer_summary.append('F')
        if hasattr(self, 'dropout'):
            layer_summary.append('+D')
        if hasattr(self, 'is_training') and self.is_training is not None:
            layer_summary.append('+BN')
            
        parameter_summary.append(''.join(layer_summary))                        
        parameter_summary.append('Q+{}bpf'.format(self.latent_bpf) if self.train_codebook else 'Q-{}bpf'.format(self.latent_bpf))
        parameter_summary.append('S+' if self.scale_latent else 'S-')
        if self.entropy_weight is not None:
            parameter_summary.append('H+{:.2f}'.format(self.entropy_weight))

        return '{}/{}'.format(super().short_name(), '-'.join(parameter_summary))        

In [ ]:
class TwitterDCN(compression.DCN):
    """
    Auto-encoder architecture described in:
    [1] L. Theis, W. Shi, A. Cunningham, and F. Huszár, “Lossy Image Compression with Compressive Autoencoders,” Mar. 2017.
    """
    
    def construct_model(self):
        
        activation = tf.nn.leaky_relu
        latent_activation = tf.nn.tanh
        last_activation = tf.nn.sigmoid
        
        self.n_layers = 9
        self.n_latent = 

        print('Building Deep Compression Network with d-latent={}'.format(self.n_latent))
        
        
        with tf.name_scope('dcn{}/normalization'.format(self.label)):
            net = 2 * (self.x - 0.5)
            print('net size: {}'.format(net.shape))

        # Convolutions        
#         n_filters = self.n_filters
        
        net = tf.contrib.layers.conv2d(net,  64, 5, stride=2, activation_fn=activation, scope='dcn{}/encoder/conv_{}'.format(self.label, 0))
        net = tf.contrib.layers.conv2d(net, 128, 5, stride=2, activation_fn=None, scope='dcn{}/encoder/conv_{}'.format(self.label, 1))
        
        resnet = tf.contrib.layers.conv2d(tf.nn.leaky_relu(net, name='dcn{}/encoder/conv_{}/lrelu'.format(self.label, 1)), 128, 3, stride=1, activation_fn=activation, scope='dcn{}/encoder/conv_{}'.format(self.label, 2))
        resnet = tf.contrib.layers.conv2d(resnet, 128, 3, stride=1, activation_fn=None, scope='dcn{}/encoder/conv_{}'.format(self.label, 3))
        net = tf.add(net, resnet, name='dcn{}/encoder/sum_a{}'.format(self.label, 0))

        resnet = tf.contrib.layers.conv2d(net, 128, 3, stride=1, activation_fn=activation, scope='dcn{}/encoder/conv_{}'.format(self.label, 4))
        resnet = tf.contrib.layers.conv2d(resnet, 128, 3, stride=1, activation_fn=None, scope='dcn{}/encoder/conv_{}'.format(self.label, 5))
        net = tf.add(net, resnet, name='dcn{}/encoder/sum_b{}'.format(self.label, 1))

        resnet = tf.contrib.layers.conv2d(net, 128, 3, stride=1, activation_fn=activation, scope='dcn{}/encoder/conv_{}'.format(self.label, 6))
        resnet = tf.contrib.layers.conv2d(resnet, 128, 3, stride=1, activation_fn=None, scope='dcn{}/encoder/conv_{}'.format(self.label, 7))
        net = tf.add(net, resnet, name='dcn{}/encoder/sum_c{}'.format(self.label, 2))
        
        net = tf.contrib.layers.conv2d(net, 96, 5, stride=2, activation_fn=None, scope='dcn{}/encoder/conv_{}'.format(self.label, 8))
        
        latent = tf.identity(net, name='dcn{}/latent'.format(self.label))
                
        inet = tf.contrib.layers.conv2d(latent, 512, 3, stride=1, activation_fn=None, scope='dcn{}/decoder/conv_{}'.format(self.label, 0))
        inet = tf.depth_to_space(inet, 2, name='dcn{}/decoder/d2s_{}'.format(self.label, 0))
        
        resnet = tf.contrib.layers.conv2d(inet, 128, 3, stride=1, activation_fn=activation, scope='dcn{}/decoder/conv_{}'.format(self.label, 1))
        resnet = tf.contrib.layers.conv2d(resnet, 128, 3, stride=1, activation_fn=None, scope='dcn{}/decoder/conv_{}'.format(self.label, 2))
        inet = tf.add(inet, resnet, name='dcn{}/decoder/sum_a{}'.format(self.label, 0))
        
        resnet = tf.contrib.layers.conv2d(inet, 128, 3, stride=1, activation_fn=activation, scope='dcn{}/decoder/conv_{}'.format(self.label, 3))
        resnet = tf.contrib.layers.conv2d(resnet, 128, 3, stride=1, activation_fn=None, scope='dcn{}/decoder/conv_{}'.format(self.label, 4))
        inet = tf.add(inet, resnet, name='dcn{}/decoder/sum_b{}'.format(self.label, 1))
        
        resnet = tf.contrib.layers.conv2d(inet, 128, 3, stride=1, activation_fn=activation, scope='dcn{}/decoder/conv_{}'.format(self.label, 5))
        resnet = tf.contrib.layers.conv2d(resnet, 128, 3, stride=1, activation_fn=None, scope='dcn{}/decoder/conv_{}'.format(self.label, 6))
        inet = tf.add(inet, resnet, name='dcn{}/decoder/sum_c{}'.format(self.label, 2))

        inet = tf.contrib.layers.conv2d(inet, 256, 3, stride=1, activation_fn=activation, scope='dcn{}/decoder/tconv_{}'.format(self.label, 7))
        inet = tf.depth_to_space(inet, 2, name='dcn{}/decoder/d2s_{}'.format(self.label, 7))
        
        inet = tf.contrib.layers.conv2d(inet, 12, 3, stride=1, activation_fn=None, scope='dcn{}/decoder/tconv_{}'.format(self.label, 8))
        inet = tf.depth_to_space(inet, 2, name='dcn{}/decoder/d2s_{}'.format(self.label, 8))
        
        with tf.name_scope('dcn{}/denormalization'.format(self.label)):
            y = (inet + 1) / 2
            
        y = tf.identity(y, name="y")

        with tf.name_scope('dcn{}/optimization'.format(self.label)):
            lr = tf.placeholder(tf.float32, name='dcn_learning_rate')
            loss = tf.nn.l2_loss(self.x - y)
            adam = tf.train.AdamOptimizer(learning_rate=lr)
            opt = adam.minimize(loss, var_list=self.parameters)
            
        return y, lr, loss, adam, opt, latent

In [ ]:
class ResDCN(compression.DCN):
    
    def construct_model(self):
        
        with tf.name_scope('dcn'):

            activation = tf.nn.ResDCN
            last_activation = tf.nn.sigmoid

            print('Building Deep Compression Network with d-latent={}'.format(n_latent))

            net = self.x
            print('net size: {}'.format(net.shape))
        
            # Convolutions
            n_filters = self.n_filters
            
            net = tf.contrib.layers.conv2d(net, 64, self.kernel, stride=2, activation_fn=activation, scope='dcn{}/conv_{}'.format(self.label, 0))
            
            for r in range(self.n_layers):
                net = tf.contrib.layers.conv2d(net, n_filters, self.kernel, stride=2, activation_fn=activation, scope='dcn{}/conv_{}'.format(self.label, r))
            #     print('net size: {}'.format(net.shape))
#                     net = tf.contrib.layers.max_pool2d(net, 2, scope='dcn{}/pool_{}'.format(self.label, r))
                print('net size: {} // {}'.format(net.shape, net))
                n_filters *= self.n_fscale

            # Flatten and get latent representation
            flat = tf.contrib.layers.flatten(net, scope='dcn{}/flat_{}'.format(self.label, 0))
            print('net size: {}'.format(flat.shape))

            latent = tf.contrib.layers.fully_connected(flat, self.n_latent, activation_fn=activation, scope='dcn{}/dense_{}'.format(self.label, 0))
            print('net size: {}'.format(latent.shape))

            inet = tf.contrib.layers.fully_connected(latent, int(flat.shape[-1]), activation_fn=activation, scope='dcn{}/dense_{}'.format(self.label, 1))
            print('net size: {}'.format(inet.shape))
            inet = tf.reshape(net, tf.shape(net), name='dcn{}/reshape_{}'.format(self.label, 0))
            print('net size: {}'.format(inet.shape))

            # Transposed convolutions
            for r in range(self.n_layers):
                inet = tf.contrib.layers.conv2d_transpose(inet, 3 if r == self.n_layers - 1 else n_filters, self.kernel, stride=2, 
                                                          activation_fn=last_activation if r == self.n_layers - 1 else activation,
                                                          scope='dcn{}/tconv_{}'.format(self.label, r))
                print('net size: {}'.format(inet.shape))
                n_filters = n_filters // self.n_fscale

            y = inet

        with tf.name_scope('dcn{}_optimization'.format(self.label)):
            lr = tf.placeholder(tf.float32, name='dcn_learning_rate')
            loss = tf.nn.l2_loss(self.x - y)
            adam = tf.train.AdamOptimizer(learning_rate=lr)
            opt = adam.minimize(loss, var_list=self.parameters)
            
        return y, lr, loss, adam, opt, latent

## Create DCN instance

In [ ]:
z_spatial = int(dcn.patch_size / (2**dcn.n_layers))
z_filters = int(dcn.n_filters * (dcn.n_fscale**(dcn.n_layers-1)))
shape = (None, z_spatial, z_spatial, z_filters)
print(shape)
# alphas = tf.get_variable('dcn{}/latent_scaling'.format(self.label), shape=(None, z_spatial, z_spatial, z_filters), dtype=tf.float32, initializer=tf.ones_initializer)

alphas = tf.get_variable('dcn{}/latent_scaling'.format(dcn.label), shape=(None, 100), dtype=tf.float32, initializer=tf.ones_initializer)

In [ ]:
graph = tf.Graph()
sess = tf.Session(graph=graph)

# dcn = AutoencoderDCN(sess, graph, patch_size=training['patch_size'], n_latent=0, n_layers=3, n_fscale=2, n_filters=8, dropout=True)
dcn = AutoencoderDCN(sess, graph, patch_size=training['patch_size'], n_latent=0, n_filters=8, n_fscale=2, n_layers=3, r_layers=0, 
                     dropout=True, is_training=True, train_codebook=True, latent_bpf=6, scale_latent=False, entropy_weight=None)

# dcn = TwitterDCN(sess, graph, patch_size=128)

print(dcn.summary())
print(dcn.short_name())
# print(dcn.count_parameters_breakdown())
print('Compression stats:', dcn.compression_stats(n_latent_bytes=0.5))

# dcn.load_model('./data/raw/compression/aedcn/8x8x192')
# dcn.save_model(os.path.join('./data/raw/compression/', dcn.short_name()), 3767)

In [ ]:
from helpers import tf_helpers
tf_helpers.show_graph(dcn.graph.as_graph_def())

## Training

In [ ]:
# dcn.init()
out = dcn.sess.run(dcn.weights, feed_dict={dcn.x: batch_x})
# fig = plotting.imsc(out[0:32, :])

with dcn.graph.as_default():
    histogram = dcn.sess.run(dcn.histogram, feed_dict={dcn.x: batch_x})
    entropy = dcn.sess.run(- tf.reduce_sum(dcn.histogram * tf.log(dcn.histogram)) / 0.6931, feed_dict={dcn.x: batch_x})

plt.plot(histogram)
print(entropy)

print(- np.sum(histogram * np.log2(histogram)))

In [ ]:
dcn.init()

# Compute the number of available batches
n_batches = data['training']['y'].shape[0] // training['batch_size']
v_batches = data['validation']['y'].shape[0] // training['batch_size']

# Structures for storing performance stats
perf = {
    'loss': {'training': [], 'validation': []},
    'entropy': {'training': [], 'validation': []},
    'ssim': {'training': [], 'validation': []}
}

caches = {
    'loss': {'training': deque(maxlen=n_batches), 'validation': deque(maxlen=v_batches)},
    'entropy': {'training': deque(maxlen=n_batches), 'validation': deque(maxlen=v_batches)},
    'ssim': {'training': deque(maxlen=n_batches), 'validation': deque(maxlen=v_batches)}
}

# Configure data augmentation
augmentation_probs = {
    'resize': 0.0,
    'flip_h': 0.5,
    'flip_v': 0.5
}

sample_dropout = False
learning_rate = 1e-4
sampling_rate = 100

model_output_dirname = os.path.join('./data/raw/compression/', dcn.short_name())

# Create a summary writer and create the necessary directories
sw = dcn.get_summary_writer(model_output_dirname)

with tqdm.tqdm(total=training['n_epochs'], ncols=160, desc='Train') as pbar:

    for epoch in range(0, training['n_epochs']):
        
        if epoch> 0 and epoch % 1000 == 0:
            learning_rate = learning_rate / 2

        # Iterate through batches of the training data 
        for batch_id in range(n_batches): # TODO n_batches
            
            # Pick random patch size - will be resized later for augmentation
            current_patch = np.random.choice(np.arange(training['patch_size'], 2 * training['patch_size']), 1) if np.random.uniform() < augmentation_probs['resize'] else training['patch_size']
            
            # Sample next batch
            batch_x = data.next_training_batch(batch_id, training['batch_size'], current_patch)
            
            # If rescaling needed, apply
            if training['patch_size'] != current_patch:
                batch_t = np.zeros((batch_x.shape[0], training['patch_size'], training['patch_size'], 3), dtype=np.float32)
                for i in range(len(batch_x)):
                    batch_t[i] = resize(batch_x[i], [training['patch_size'], training['patch_size']], anti_aliasing=True)
                batch_x = batch_t                
            
            # Data augmentation - random horizontal flip
            if np.random.uniform() < augmentation_probs['flip_h']: batch_x = batch_x[:, :, ::-1, :]
            if np.random.uniform() < augmentation_probs['flip_v']: batch_x = batch_x[:, ::-1, :, :]
            
            # Sample dropout
            keep_prob = 1.0 if not sample_dropout else np.random.uniform(0.5, 1.0)            
            
            # Make a training step
            values = dcn.training_step(batch_x, learning_rate, dropout_keep_prob=keep_prob)
            for key, value in values.items():
                caches[key]['training'].append(value)                
            
        # Record average values for the whole epoch
        for key in ['loss', 'ssim', 'entropy']:
            perf[key]['training'].append(np.mean(caches[key]['training']))

        # Get some extra stats
        if dcn.scale_latent:
            scaling = dcn.sess.run(dcn.graph.get_tensor_by_name('dcn/latent_scaling:0'))
        else:
            scaling = np.nan
            
        codebook = dcn.sess.run(dcn.codebook)        

        # Iterate through batches of the validation data
        if epoch % sampling_rate == 0:
            for batch_id in range(v_batches): # TODO v_batches
                batch_x = data.next_validation_batch(batch_id, training['batch_size'], training['patch_size'])
                batch_y = dcn.process(batch_x)
                
                # Compute loss
                loss_value = np.linalg.norm(batch_x - batch_y)
                caches['loss']['validation'].append(loss_value)                
                
                # Compute SSIM            
#                 ssim_value = np.mean([ssim((255*batch_x[r]).astype(np.uint8), (255*batch_y[r]).astype(np.uint8)) for r in range(len(batch_x))])
                ssim_value = np.mean([ssim(batch_x[r], batch_y[r], multichannel=True) for r in range(len(batch_x))]) 
                caches['ssim']['validation'].append(ssim_value)

            perf['loss']['validation'].append(np.mean(caches['loss']['validation']))
            perf['ssim']['validation'].append(np.mean(caches['ssim']['validation']))
            
            # Save current snapshot
            thumbs = (255 * plotting.thumbnails(np.concatenate((batch_x[::2], batch_y[::2]), axis=0), n_cols=20)).astype(np.uint8)
            thumbs_few = (255 * plotting.thumbnails(np.concatenate((batch_x[::10], batch_y[::10]), axis=0), n_cols=4)).astype(np.uint8)
            imageio.imsave(os.path.join(model_output_dirname, 'thumbnails-{:05d}.png'.format(epoch)), thumbs)
            
            # Sample latent space
            batch_z = dcn.compress(batch_x) # data.next_validation_batch(0, 256)
        
            # Save summaries to TB            
            summary = tf.Summary()
            summary.value.add(tag='loss/validation', simple_value=perf['loss']['validation'][-1])
            summary.value.add(tag='loss/training', simple_value=perf['loss']['training'][-1])
            summary.value.add(tag='ssim/validation', simple_value=perf['ssim']['validation'][-1])
            summary.value.add(tag='ssim/training', simple_value=perf['ssim']['training'][-1])
            summary.value.add(tag='entropy/training', simple_value=perf['entropy']['training'][-1])
            summary.value.add(tag='codebook/min', simple_value=codebook.min())
            summary.value.add(tag='codebook/max', simple_value=codebook.max())
            summary.value.add(tag='codebook/mean', simple_value=codebook.mean())
            summary.value.add(tag='codebook/diff_variance', simple_value=np.var(np.convolve(code_book, [-1, 1], mode='valid')))
            summary.value.add(tag='scaling', simple_value=scaling)
            summary.value.add(tag='images/reconstructed', image=summaries.log_image(rescale(thumbs_few, 1.0, anti_aliasing=True)))
            summary.value.add(tag='histograms/latent', histo=summaries.log_histogram(batch_z))
            sw.add_summary(summary, epoch)
            sw.flush()        

        # Update progress bar
        pbar.set_postfix(L=np.mean(perf['loss']['training'][-3:]), 
                         Lv=np.mean(perf['loss']['validation'][-1:]),
#                          lr='{:.8f}'.format(learning_rate),
                         ssim=perf['ssim']['validation'][-1],
                         H=np.mean(perf['entropy']['training'][-1:]), 
                         S=scaling,
                         Qvar=np.var(np.convolve(code_book, [-1, 1], mode='valid'))
                        )
        pbar.update(1)

### Save model

In [ ]:
dcn.save_model(model_output_dirname, epoch)

In [ ]:
import matplotlib.pyplot as plt
from helpers import utils

fig = plt.figure(figsize=(20, 4))
ax = fig.gca()
ax.plot(utils.ma_conv(perf['loss']['training'], n=11))
# ax.plot(np.arange(0, len(loss['training']), sampling_rate), utils.ma_conv(loss['validation'], n=3))
ax.plot(np.arange(0, len(perf['loss']['training']), sampling_rate), perf['loss']['validation'], '-o', alpha=0.3)
ax.plot(perf['loss']['training'], '.', alpha=0.1)
ax.legend(['train', 'valid'], loc='upper right')
# ax.set_yscale('log')

In [ ]:
dcn.load_model(os.path.join('./data/raw/compression/', dcn.short_name()))

In [ ]:
# Show a sample and a reconstruction of the current batch
batch_y = dcn.process(batch_x)
f = plotting.imsc(batch_x[0:8], ncols=8, figwidth=20)
f = plotting.imsc(batch_y[0:8], ncols=8, figwidth=20)

## Explore & Understand the Latent Representation

In [ ]:
# Estimate entropy
sample_batch_size = 400

batch_x = data.next_validation_batch(0, sample_batch_size)

# See latent distribution
batch_z = dcn.compress(batch_x)
batch_z = batch_z.reshape((sample_batch_size, -1)).T
print(batch_z.shape)

feature_id = 1

# data = batch_z[feature_id]

n_bins = 128
bin_boundaries = np.linspace(-32, 32, n_bins+1)
bin_centers = np.convolve(bin_boundaries, [0.5, 0.5], mode='valid')

# for bin, value in zip(bin_centers, hist):
#     print(bin, '->', value)   

hist = np.histogram(batch_z[:], bins=bin_boundaries, normed=True)[0]

fig = plt.figure()
ax = fig.gca()
ax.bar(bin_centers, hist, width=bin_centers[1] - bin_centers[0], color='r')
ax.set_title('Histogram of quantized coefficients')

In [ ]:
# q_min = np.percentile(batch_z.astype(np.int32), 0.1)
# q_max = np.percentile(batch_z.astype(np.int32), 100 - 0.1)

q_min = np.percentile(batch_z.astype(np.int32), 0.1)
q_max = np.percentile(batch_z.astype(np.int32), 100 - 0.1)


bin_boundaries = np.arange(q_min + 0.5, q_max + 0.5, 1)
bin_centers = np.convolve(bin_boundaries, [0.5, 0.5], mode='valid')

print('Bin centers ({}): {}'.format(len(bin_centers), bin_centers.tolist()))

In [ ]:
# value = np.array([[-3.1231]])
batch_ex = batch_z.T[0]
value = batch_ex.reshape((4096, 1))

sigma = 3
weights = np.exp(-sigma * np.power(value - bin_centers.reshape(1, len(bin_centers)), 2))
weights = weights / weights.sum(axis=1, keepdims=True)

feature_id = 20
print('Feature', feature_id)
print('Value', value[feature_id])
print('Quantization', np.round(value[feature_id]))
print('Soft quantization', np.sum(weights[feature_id] * bin_centers))

# 
plt.plot(bin_centers, weights[feature_id])
plt.title('{} / {}'.format(batch_ex[feature_id], np.sum(soft[feature_id])))

# plt.plot(bin_centers, weights.squeeze())
# plt.title('{} / {}'.format(value, np.sum(weights)))

In [ ]:
# Compute frequencies using unique
indices, _ = vq(batch_z.reshape((-1, )), bin_centers)
unique, counts = np.unique(bin_centers[indices], return_counts=True)
counts = counts / counts.sum()

# Compute soft histogram
histogram = np.mean(weights, axis=0).clip(1e-6)
histogram = histogram / np.sum(histogram)

entropy_estimate = - np.sum(histogram * np.log2(histogram))

plt.plot(unique, counts)
plt.plot(bin_centers, histogram)
plt.legend(['unique', 'soft'])
plt.title('Soft-estimated entropy {:.2f}'.format(entropy_estimate))

## Entropy Coding

In [ ]:
q_min = np.percentile(batch_z.astype(np.int32), 0.1)
q_max = np.percentile(batch_z.astype(np.int32), 100 - 0.1)

bin_boundaries = np.arange(q_min + 0.5, q_max - 0.5, 1)
bin_centers = np.convolve(bin_boundaries, [0.5, 0.5], mode='valid')

# print(bin_boundaries)

# Compute frequencies using a histogram
freq = np.histogram(batch_z[:], bins=bin_boundaries, normed=False)[0]
hist = np.histogram(batch_z[:], bins=bin_boundaries, normed=True)[0]
hist = hist.clip(1e-9)
probs = hist / np.sum(hist)

# Compute frequencies using unique
indices, _ = vq(batch_z.reshape((-1, )), bin_centers)
unique, counts = np.unique(bin_centers[indices], return_counts=True)

print('Bin centers - {} values'.format(len(bin_centers)))
for bin, value in zip(bin_centers, freq):
    print('  ',bin, '->', value.round(2))   

# print(hist.round(2))
# print(probs.round(2))

entropy = np.sum(- probs * np.log2(probs))

print('Naive coding: {:.2f}'.format(np.log2(len(bin_centers))))
print('Entropy: {:.2f}'.format(entropy))

from dahuffman import HuffmanCodec

codec = HuffmanCodec.from_frequencies({k: v for k, v in zip(bin_centers.astype(np.int), freq)})

In [ ]:
{k: v for k, v in zip(bin_centers.astype(np.int), probs)}

In [ ]:
for word codec.get_code_table()

In [ ]:
from scipy.cluster.vq import vq

print(batch_z.T[0].shape)

code_book = bin_centers.astype(np.int)

# Vector quantization
indices, distortion = vq(batch_z.T[0], code_book)
batch_q = code_book[indices]

print('Codebook {}: {}'.format(len(code_book), code_book))
print(batch_z.T[0])
print(batch_q)
print(indices)

encoded = codec.encode(batch_q)
bits_per_symbol = np.ceil(np.log2(len(bin_centers)))

print('Naive coding: {:.0f} bytes'.format(bits_per_symbol * len(batch_q) / 8))
print('Theoretical limit: {:.0f} bytes'.format(entropy * len(batch_q) / 8))
print('Compressed (Huffman): {} bytes'.format(len(encoded)))

In [ ]:
bin_centers

In [ ]:
codec.print_code_table()

In [ ]:
model_output_dirname = os.path.join('./data/raw/compression/', dcn.short_name(), 'raise')
dcn.load_model(model_output_dirname)

In [ ]:
from helpers import plotting

# dcn.init()

batch_x = data.next_validation_batch(0, 256)

# See latent distribution
# batch_z = dcn.compress(batch_x).T
batch_z = dcn.compress(batch_x).reshape((256, -1)).T
# batch_z = batch_x.reshape((256, 128*128*3)).T
print(batch_z.shape)

# cov = batch_z.T * batch_z

# cov = np.cov(batch_z)

cov = np.corrcoef(batch_z)

plt.imshow(cov)

# batch_z = batch_z[:9]

# fig, axes = plotting.sub(len(batch_z) + 1, ncols=10, figwidth=20)

# # print(axes)

# for i, ax in enumerate(axes):
   
#     if i >= len(batch_z):
#         plotting.quickshow(cov)
#     else:        
#         ax.hist(batch_z[i], bins=30)
#         ax.set_yticks([])

In [ ]:
bin_centers = [-3, -2, -1, 0, 1, 2, 3]
p, x = np.histogram(np.random.normal(size=(1000,)), bins=bin_centers)
print(bin_centers)
print(x)
print(np.convolve(x, [0.5, 0.5], mode='valid'))
print(p)
plt.plot(np.convolve(x, [0.5, 0.5], mode='valid'), p)

In [ ]:
latent_means = np.mean(batch_z,axis=1)
fig, axes = plotting.sub(2, figwidth=12)
axes[0].plot(latent_means)
axes[0].set_title('Mean values for all latent variables')
axes[1].hist(latent_means, bins=50, normed=True)
axes[1].set_title('Histogram of mean values for all latent variables')

In [ ]:
batch_x = data.next_validation_batch(0, 400)

batch_z = dcn.compress(batch_x)
batch_z = batch_z.reshape((sample_batch_size, -1)).T
print(batch_z.shape)

n_bins = 64
bin_boundaries = np.linspace(-16, 16, n_bins+1)
bin_centers = np.convolve(bin_boundaries, [0.5, 0.5], mode='valid')

print(bin_centers)

distribution = np.zeros((len(batch_z), n_bins))
for i in range(len(batch_z)):
    distribution[i, :] = np.histogram(batch_z[i], bins=bin_boundaries, normed=True)[0]

vis = []
for i in range(len(distribution) // 512):
    vis.append(distribution[i*512:(i+1)*512, :])
    

thumbs = plotting.thumbnails(vis, n_cols=len(vis))
    
fig = plotting.imsc(thumbs, 'A', figwidth=20)

In [ ]:
n_bins = 64
bin_boundaries = np.linspace(-256, 256, n_bins+1)
bin_centers = np.convolve(bin_boundaries, [0.5, 0.5], mode='valid')

i = 58
fig = plt.figure(figsize=(16, 4))
fig.gca().fill_between(bin_centers, 0, np.histogram(batch_z[i], bins=bin_boundaries, normed=True)[0], alpha=0.1)
print(distribution[i, :])
print(batch_z[i])

In [ ]:
fig = plt.figure(figsize=(16, 4))
for i in range(len(batch_z)):
#     fig.gca().plot(bin_centers, distribution[i, :])
    fig.gca().fill_between(bin_centers, 0, distribution[i, :], alpha=0.1)

In [ ]:
for op in dcn.sess.graph.get_operations():
    print(op.name)

## Quantization of the latent space

In [ ]:
from scipy.cluster.vq import vq

code_book = np.array([-1, 0, 1])

batch_x = data.next_validation_batch(0, 256)

alpha = 1
n_unique = 64

# See latent distribution
# batch_z = dcn.compress(batch_x).T
batch_z = dcn.compress(batch_x)
# print(batch_z[0, 0:15].round(3))
# batch_z = batch_z.clip(-100, 100)
# batch_z = (alpha * batch_z).astype(np.int32) / alpha
# batch_z = ((alpha * batch_z).astype(np.int32).clip(-n_unique // 2 + 1, n_unique // 2) / alpha)
# print(batch_z[0, 0:15])
print('Int from {} to {}'.format(-n_unique // 2 + 1, n_unique // 2))
print('  unique = {}'.format(np.unique(batch_z)))
print('# unique = {}'.format(len(np.unique(batch_z))))

# Vector quantization
# indices, distortion = vq(batch_z, code_book)
# batch_z = code_book[indices]

batch_y = dcn.decompress(batch_z)

f = plotting.imsc(batch_x[0:8], ncols=8, figwidth=20)
f = plotting.imsc(batch_y[0:8], ncols=8, figwidth=20)

In [ ]:
dim_id = 0
fig = plt.figure(figsize=(8, 8))
fig.gca().hist(batch_z[dim_id])
print(batch_z[dim_id].shape)

In [ ]:
dcn.save_model('./data/raw/compression/aedcn/8x8x192', epoch)

## Save the actual bitstream

In [ ]:
q_min = -16
q_max = 16

In [ ]:
graph = tf.Graph()
sess = tf.Session(graph=graph)

dcn = AutoencoderDCN(sess, graph, patch_size=training['patch_size'], n_latent=0, n_layers=3, n_fscale=2, n_filters=16, dropout=0.5, is_training=True)
# dcn = AutoencoderDCN(sess, graph, patch_size=512, n_latent=0, n_layers=3, n_fscale=2, n_filters=16, dropout=0.5, is_training=True)
# model_output_dirname = os.path.join('./data/raw/compression/', dcn.short_name(), '128x128_bn_mult_no_entropy')
dcn.load_model(model_output_dirname)

In [ ]:
'dcn/encoder/bn_0/moving_mean'
dcn.parameters
with dcn.graph.as_default():
    for tv in [tv for tv in tf.trainable_variables()]:
        print(tv)

In [ ]:
from scipy.cluster.vq import vq

max_value = 32

qmin = -max_value
qmax = max_value + 2
step = 1

code_book = np.arange(qmin, qmax, step)
code_book_edges = np.convolve(code_book, [0.5, 0.5], mode='valid')
code_book = code_book[1:-1]

image = imageio.imread('./data/clic/alberto-montalesi-176097.png').astype(np.float32) / (2**8 - 1)
batch_x = utils.slidingwindow(image, 128)[6:7]

# See latent distribution
# batch_z = dcn.compress(batch_x).T
batch_z = dcn.compress(batch_x)

print('Codebook size: {} // {}'.format(len(code_book), code_book.tolist()))

# Compute frequencies using unique
indices, _ = vq(batch_z.reshape((-1, )), code_book)
counts = np.histogram(code_book[indices], bins=code_book_edges)[0]

plt.plot(code_book, counts)

# Construct a Huffman codec
codec = HuffmanCodec.from_frequencies({k: v for k, v in zip(code_book.astype(np.int), counts)})

# Vector quantization
indices, distortion = vq(batch_z.reshape((-1)), code_book)
batch_q = code_book[indices]

# Zero some elements

# batch_q[np.random.uniform(size=batch_q.shape) > 0.1] = 0

batch_y = dcn.decompress(batch_q.reshape(dcn.latent_shape))

coded_image = codec.encode(batch_q.tolist())

ssim_value = ssim(batch_x[0], batch_y[0], multichannel=True)

counts = counts.clip(min=1)
probs = counts / counts.sum()
entropy = - np.sum(probs * np.log2(probs))

print('Pixels          : {}x{} = {:,}'.format(128, 128, 128*128))
print('Bitmap          : {:,} bytes'.format(128*128*3))
print('Batch size      : {:,} elements'.format(np.prod(batch_x.shape)))
print('Code-book size  : {} elements'.format(len(code_book)))
print('Entropy         : {:.2f} bits per symbol'.format(entropy))
print('Latent size     : {:,}'.format(np.prod(batch_z.shape)))
print('PPF Naive       : {:,.0f} --> {:,.0f} bytes [{} bits per element]'.format(np.prod(batch_z.shape) * np.log2(len(code_book)) / 8,
                                               np.prod(batch_z.shape) * np.ceil(np.log2(len(code_book))) / 8,
                                               np.ceil(np.log2(len(code_book)))
                                              ))
print('PPF Theoretical : {:,.0f} bytes'.format(np.prod(batch_z.shape) * entropy / 8))
print('PPF Huffman     : {:,} bytes ({:.2f} bpp) --> ssim: {:.2f}'.format(len(coded_image), 
                                                              8 * len(coded_image) / np.prod(batch_x.shape[1:3]),
                                                              ssim_value,
                                                             ))
# Encode JPEG
s = io.BytesIO()
imageio.imsave(s, (255*batch_x).astype(np.uint8).squeeze(), format='jpg', quality=20)
# imageio.imsave('test.jpg', (255*batch_x).astype(np.uint8).squeeze(), format='jpg', quality=50)
image_compressed = imageio.imread(s.getvalue())
batch_j = np.expand_dims(image_compressed, axis=0) / (2**8 - 1)

ssim_value = ssim(batch_x[0], batch_j[0], multichannel=True)

print('JPEG stream     : {:,} bytes ({:0.2f} bpp) --> ssim: {:.2f}'.format(len(s.getvalue()), 8 * len(s.getvalue()) / np.prod(batch_j.shape[1:3]), ssim_value ))

f = plotting.imsc(np.concatenate((batch_x[0:8], batch_y[0:8], batch_j[0:8]), axis=0), ncols=3 * len(batch_x), figwidth=14)